<a href="https://colab.research.google.com/github/eh1k4ri/Ponderada-Cartpole/blob/main/implementacao-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importar bibliotecas necessárias
%pip install gym[classic_control] matplotlib
import numpy as np
import matplotlib.pyplot as plt
import gym

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Configuração do Ambiente
env = gym.make('CartPole-v1', render_mode='human')

In [ ]:
# Parâmetros de discretização
n_bins = 24  # Número de divisões em cada dimensão do estado
bins = [
    np.linspace(-4.8, 4.8, n_bins),        # Posição do carrinho
    np.linspace(-5, 5, n_bins),            # Velocidade do carrinho
    np.linspace(-0.418, 0.418, n_bins),    # Ângulo do pêndulo
    np.linspace(-5, 5, n_bins)             # Velocidade angular do pêndulo
]

In [ ]:
# Função para discretizar estados contínuos
def discretize_state(state):
    state_index = []
    for i, val in enumerate(state):
        state_index.append(np.digitize(val, bins[i]) - 1)
    return tuple(state_index)

In [ ]:

# Inicializar a tabela Q com valores aleatórios
action_space_size = env.action_space.n  # Número de ações possíveis (2: esquerda ou direita)
state_space_size = [n_bins] * len(bins)  # Dimensões do espaço de estados discretizado
Q_table = np.random.uniform(low=-1, high=1, size=state_space_size + [action_space_size])

In [ ]:
# Definição dos hiperparâmetros
alpha = 0.1          # Taxa de aprendizado
gamma = 0.99         # Fator de desconto
epsilon = 1.0        # Taxa de exploração inicial
epsilon_decay = 0.995  # Decaimento da taxa de exploração
epsilon_min = 0.01    # Valor mínimo de epsilon
num_episodes = 10000  # Número total de episódios para treinamento

In [ ]:
# Função para escolher ação (política epsilon-gulosa)
def choose_action(state):
    if np.random.random() < epsilon:
        # Exploração: escolhe uma ação aleatória
        return np.random.choice(action_space_size)
    else:
        # Exploração: escolhe a ação com maior valor Q no estado atual
        return np.argmax(Q_table[state])

In [ ]:
# Loop de treinamento
rewards = []  # Lista para armazenar as recompensas de cada episódio

for episode in range(num_episodes):
    # Reinicialização do ambiente e discretização do estado inicial
    state_raw = env.reset()  # Ajuste para se adaptar à versão do Gym
    if isinstance(state_raw, tuple):
        state_raw = state_raw[0]  # Pegue apenas o estado, se o retorno for uma tupla
    state = discretize_state(state_raw)
    done = False
    total_reward = 0

    while not done:
        # Escolher ação com base na política epsilon-gulosa
        action = choose_action(state)

        # Executar a ação e obter o próximo estado, recompensa e status
        step_result = env.step(action)

        # Ajuste para lidar com diferentes versões do Gym que retornam tupla com mais de 4 elementos
        if len(step_result) == 4:
            next_state_raw, reward, done, _ = step_result
        elif len(step_result) == 5:  # Para versões mais novas que retornam (state, reward, done, truncated, info)
            next_state_raw, reward, done, truncated, _ = step_result
            done = done or truncated
        else:
            raise ValueError("O retorno do step tem um número inesperado de elementos.")

        next_state = discretize_state(next_state_raw)

        # Escolher a melhor ação futura (maximiza Q)
        best_next_action = np.argmax(Q_table[next_state])

        # Atualizar a tabela Q usando a Equação de Bellman
        td_target = reward + gamma * Q_table[next_state][best_next_action]
        td_delta = td_target - Q_table[state][action]
        Q_table[state][action] += alpha * td_delta

        # Atualizar o estado atual
        state = next_state
        total_reward += reward

    # Armazenar a recompensa total do episódio
    rewards.append(total_reward)

    # Decaimento da taxa de exploração epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay


/home/esthernunes/.local/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [ ]:
# Avaliação do desempenho do agente
plt.plot(range(num_episodes), rewards)
plt.xlabel('Episódio')
plt.ylabel('Recompensa Total')
plt.title('Treinamento do Agente CartPole - Q-learning')
plt.show()

In [ ]:
# Demonstração do comportamento do agente treinado
state_raw = env.reset()  # Ajuste para se adaptar à versão do Gym
if isinstance(state_raw, tuple):
    state_raw = state_raw[0]  # Pegue apenas o estado, se o retorno for uma tupla
state = discretize_state(state_raw)
done = False

while not done:
    # Escolher a ação com maior valor Q no estado atual
    action = np.argmax(Q_table[state])

    # Executar a ação no ambiente
    step_result = env.step(action)

    # Ajuste para lidar com diferentes versões do Gym
    if len(step_result) == 4:
        state_raw, _, done, _ = step_result
    elif len(step_result) == 5:  # Para versões mais novas que retornam (state, reward, done, truncated, info)
        state_raw, _, done, truncated, _ = step_result
        done = done or truncated
    else:
        raise ValueError("O retorno do step tem um número inesperado de elementos.")

    state = discretize_state(state_raw)
    env.render()  # Renderiza o ambiente para visualização

env.close()  # Fecha o ambiente após a demonstração
